In [3]:
import pandas as pd     
import seaborn as sns 

#Importando dados
clientes = pd.read_csv('clientes.csv')

print(clientes.shape)

print(clientes.describe())

#Substituindo por número
clientes['sexo']= clientes['sexo'].map({'Male':0, 'Female':1})
clientes['estado_civil']= clientes['estado_civil'].map({'No':0, 'Yes':1})
clientes['aprovacao_emprestimo']= clientes['aprovacao_emprestimo'].map({'N':0, 'Y':1})

#Tirar valores nulos
clientes = clientes.dropna()

X = clientes[['sexo', 'estado_civil', 'renda', 'emprestimo', 'historico_credito']]
y = clientes.aprovacao_emprestimo
X.shape, y.shape

#Importando amostragem de dados
from sklearn.model_selection import train_test_split

# Amostragem
x_train, x_teste, y_train, y_teste = train_test_split(X,y, test_size = 0.2)


## Algoritmo Random Forest
from sklearn.ensemble import RandomForestClassifier 

maquina = RandomForestClassifier() 
maquina.fit(x_train, y_train)

#Importando a Métrica de Avaliação
from sklearn.metrics import accuracy_score

#Avaliando a Acurácia com dados de treino
pred_maquina = maquina.predict(x_train)
accuracy_score(y_train,pred_maquina)


#Avaliando a Acurácia da Máquina Preditiva Baseline com dados de Teste
pred_maquina = maquina.predict(x_teste)
accuracy_score(y_teste,pred_maquina)


#Criando hiperparâmetro (maxdepth)
maquina = RandomForestClassifier(max_depth=4) 
maquina.fit(x_train, y_train)

#Avaliando a Acurácia com os dados de Treino
pred_maquina_treino = maquina.predict(x_train)
accuracy_score(y_train,pred_maquina_treino)


#Avaliando a Acurácia com dados de Teste
pred_maquina_teste = maquina.predict(x_teste)
accuracy_score(y_teste,pred_maquina_teste)


#Importância das Variáveis
maquina.feature_importances_


#Gráfico de importância 
print(maquina.feature_importances_)

from matplotlib import pyplot
pyplot.bar(range(len(maquina.feature_importances_)), maquina.feature_importances_)
pyplot.show()

#['sexo', 'estado_civil', 'renda', 'emprestimo', 'historico_credito']

#Salvando Máquina Preditiva
import pickle 
pickle_out = open("maquina_preditiva.pkl", mode = "wb") 
pickle.dump(maquina, pickle_out) 
pickle_out.close()

In [ ]:
!pip install -q pyngrok
!pip install -q streamlit
!pip install -q streamlit_ace

In [5]:
#Comando para Criar o Sistema no arquivo app.py
%%writefile app.py

import pickle
import streamlit as st
 
# Carregando a Máquina Preditiva
pickle_in = open('maquina_preditiva.pkl', 'rb') 
maquina_preditiva = pickle.load(pickle_in)

#Manter a sessão em cache 
@st.cache()
  
# Criando a função que irá fazer a predição usando os dados impostados pelo usuário do Sistema 
def prediction(sexo, estado_civil, renda, emprestimo, historico_credito):   
 
    # Pre-processando a entrada do Usuário    
    if sexo == "Masculino":
        sexo = 0
    else:
        sexo = 1
 
    if estado_civil == "Solteiro(a)":
        estado_civil = 0
    else:
        estado_civil = 1
 
    if historico_credito == "Débitos Pendentes":
        historico_credito = 0
    else:
        historico_credito = 1  
 
    emprestimo = emprestimo / 1000
 
    # Predições
    prediction = maquina_preditiva.predict( 
        [[sexo, estado_civil, renda, emprestimo, historico_credito]])
     
    if prediction == 0:
        pred = 'Rejeitado'
    else:
        pred = 'Aprovado'
    return pred
      
  
# Essa função é para criação da webpage  
def main():  

    # Elementos da webpage
    html_temp = """ 
    <div style ="background-color:blue;padding:13px"> 
    <h1 style ="color:white;text-align:center;">SAE</h1> 
    <h2 style ="color:white;text-align:center;">Sistema de Aprovação de Empréstimos</h2> 
    </div> 
    """
      
    # Stramlit que faz o display
    st.markdown(html_temp, unsafe_allow_html = True) 
      
    # Caixas de respostas usuário
    sexo = st.selectbox('Sexo',("Masculino","Feminino"))
    estado_civil = st.selectbox('Estado Civil',("Solteiro(a)","Casado(a)")) 
    renda = st.number_input("Renda Mensal") 
    emprestimo = st.number_input("Valor Total do Empréstimo")
    historico_credito = st.selectbox('Histórico de Créditos',("Sem Débitos","Débitos Pendentes"))
    result =""
      
    if st.button("Verificar"): 
        result = prediction(sexo, estado_civil, renda, emprestimo, historico_credito) 
        st.success('O empréstimo foi {}'.format(result))
        print(emprestimo)
     
if __name__=='__main__': 
    main()

Writing app.py


In [7]:
#Executar
!streamlit run app.py &>/dev/null&

#biblioteca para web
from pyngrok import ngrok

In [8]:
#Publica o sistema numa URL
public_url = ngrok.connect('8501')

In [9]:
#URL
public_url

<NgrokTunnel: "http://4c3a548ad4d2.ngrok.io" -> "http://localhost:8501">